In [7]:
import pandas as pd
import numpy as np
import sklearn

In [55]:
data = pd.read_csv("./data/train/01.csv")
data.head(1)

,ID,種類,地域,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,...,前面道路：方位,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log
0,1000809,中古マンション等,NaN,1101,北海道,札幌市中央区,北１条西,西１８丁目,5,オープンフロア,...,NaN,NaN,NaN,商業地域,80.0,400.0,2019年第3四半期,改装済,NaN,6.963788


# 削除したカラム

・ID:不要 \
・種類:全部同じ値のため　　\
・地域:市区町村コードで代用できると判断したため\
・都道府県名:"地域"と同様の理由\
・地区名:"地域"と同様の理由 \
・土地の形状:ほぼNULL値のため不要\
・間口:ほぼNULL値のため不要 \
・延床面積（㎡）:ほぼNULL値のため不要 \
・用途:多くの値が「住宅」を占めるため不要と判断\
・今後の利用目的:多くの値が「住宅」を占めるため不要と判断 
#### "用途"と"今後の利用目的"結合すればNULL値がなくなりそうなので要検討
・前面道路：方位:ほぼNULL値のため不要 \
・前面道路：種類:ほぼNULL値のため不要 \
・前面道路：幅員（ｍ）:ほぼNULL値のため不要\
・取引の事情等:ほぼNULL値のため不要

In [80]:
data = pd.read_csv("./data/train/01.csv").head(10)
# 確実にいらないカラムたち
data = data.drop(['ID','種類','地域', '都道府県名', '市区町村名', '地区名','土地の形状', '間口', 
                                  '延床面積（㎡）','用途','今後の利用目的','前面道路：方位', '前面道路：種類', 
                                  '前面道路：幅員（ｍ）','取引の事情等','最寄駅：名称'],axis=1) 
# もしかしたら使うかもしれないカラムたち
data = data.drop(['市区町村コード','都市計画'],axis=1)
# data = data.fillna({'改装':'未改装'})
data.rename(columns={'最寄駅：距離（分）':'nearest_station','間取り':'floor_type','面積（㎡）':'area','建築年':'builted_year',
            '建ぺい率（％）':'coverrage','容積率（％）':'floor_rate','取引時点':'sold_year','建物の構造':'structure'},inplace = True)
data

,nearest_station,floor_type,area,builted_year,structure,coverrage,floor_rate,sold_year,改装,取引価格（総額）_log
0,5,オープンフロア,85,昭和49年,ＳＲＣ,80.0,400.0,2019年第3四半期,改装済,6.963788
1,10,３ＬＤＫ,70,平成7年,ＲＣ,60.0,200.0,2013年第1四半期,未改装,7.000000
2,5,３ＬＤＫ,70,平成6年,ＲＣ,60.0,200.0,2019年第4四半期,改装済,7.301030
3,4,３ＬＤＫ,85,平成15年,ＳＲＣ,60.0,300.0,2009年第2四半期,未改装,7.255273
4,3,１Ｋ,20,昭和64年,ＳＲＣ,80.0,600.0,2017年第1四半期,未改装,6.361728
5,6,３ＬＤＫ,60,昭和52年,ＲＣ,60.0,200.0,2019年第3四半期,未改装,6.740363
6,9,３ＬＤＫ,85,平成20年,ＲＣ,60.0,200.0,2011年第1四半期,未改装,7.361728
7,8,３ＬＤＫ,55,昭和52年,ＲＣ,60.0,200.0,2016年第1四半期,NaN,6.681241
8,4,４ＬＤＫ,90,平成9年,ＳＲＣ,80.0,300.0,2008年第1四半期,未改装,7.301030
9,10,３ＬＤＫ,75,平成19年,ＲＣ,60.0,200.0,2015年第4四半期,未改装,7.397940


In [36]:
data

,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,建ぺい率（％）,容積率（％）,取引時点,改装,取引価格（総額）_log
0,5,オープンフロア,85,昭和49年,ＳＲＣ,80.0,400.0,2019年第3四半期,改装済,6.963788
1,10,３ＬＤＫ,70,平成7年,ＲＣ,60.0,200.0,2013年第1四半期,未改装,7.000000
2,5,３ＬＤＫ,70,平成6年,ＲＣ,60.0,200.0,2019年第4四半期,改装済,7.301030
3,4,３ＬＤＫ,85,平成15年,ＳＲＣ,60.0,300.0,2009年第2四半期,未改装,7.255273
4,3,１Ｋ,20,昭和64年,ＳＲＣ,80.0,600.0,2017年第1四半期,未改装,6.361728
5,6,３ＬＤＫ,60,昭和52年,ＲＣ,60.0,200.0,2019年第3四半期,未改装,6.740363
6,9,３ＬＤＫ,85,平成20年,ＲＣ,60.0,200.0,2011年第1四半期,未改装,7.361728
7,8,３ＬＤＫ,55,昭和52年,ＲＣ,60.0,200.0,2016年第1四半期,NaN,6.681241
8,4,４ＬＤＫ,90,平成9年,ＳＲＣ,80.0,300.0,2008年第1四半期,未改装,7.301030
9,10,３ＬＤＫ,75,平成19年,ＲＣ,60.0,200.0,2015年第4四半期,未改装,7.397940


### 間取りを順序尺度に変換できれば使えそう

・面積の平均値によってランクを付与したものを使用する\
・部屋数と間取りを対応させる
・

In [18]:
data.groupby('間取り').agg(s_mean=('面積（㎡）','mean')).reset_index()

,間取り,s_mean
0,オープンフロア,65.825243
1,メゾネット,110.000000
2,１ＤＫ,27.947548
3,１ＤＫ＋Ｓ,50.000000
4,１Ｋ,21.179727
5,１Ｌ,15.000000
6,１ＬＤＫ,47.845138
7,１ＬＤＫ＋Ｓ,61.250000
8,１Ｒ,21.968085
9,２ＤＫ,46.692677


In [4]:
import glob
files = glob.glob('./data/train/*')
print(files)

['./data/train/40.csv', './data/train/41.csv', './data/train/43.csv', './data/train/42.csv', './data/train/46.csv', './data/train/47.csv', './data/train/45.csv', './data/train/44.csv', './data/train/37.csv', './data/train/23.csv', './data/train/22.csv', './data/train/36.csv', './data/train/08.csv', './data/train/20.csv', './data/train/34.csv', './data/train/35.csv', './data/train/21.csv', './data/train/09.csv', './data/train/25.csv', './data/train/31.csv', './data/train/19.csv', './data/train/18.csv', './data/train/30.csv', './data/train/24.csv', './data/train/32.csv', './data/train/26.csv', './data/train/27.csv', './data/train/33.csv', './data/train/16.csv', './data/train/02.csv', './data/train/03.csv', './data/train/17.csv', './data/train/29.csv', './data/train/01.csv', './data/train/15.csv', './data/train/14.csv', './data/train/28.csv', './data/train/04.csv', './data/train/10.csv', './data/train/38.csv', './data/train/39.csv', './data/train/11.csv', './data/train/05.csv', './data/tr

In [82]:
def merge_train_data(files:list)->pd.DataFrame:
    count = 0
    for file in files:
        train_i = pd.read_csv(file)
        if (count==0):
            df = train_i.copy()
            count+=1
            continue
        df = pd.concat([df,train_i],axis = 0)
    return df
df = merge_train_data(files)

def data_processing(df:pd.DataFrame)->pd.DataFrame:
    # 確実にいらないカラムたち
    data = df.copy()
    data = data.drop(['ID','種類','地域', '都道府県名', '市区町村名', '地区名','土地の形状', '間口', 
                                  '延床面積（㎡）','用途','今後の利用目的','前面道路：方位', '前面道路：種類', 
                                  '前面道路：幅員（ｍ）','取引の事情等','最寄駅：名称'],axis=1) 
    # もしかしたら使うかもしれないカラムたち
    data = data.drop(['市区町村コード','都市計画'],axis=1)
    data = data.fillna({'改装':'未改装'})
    data.rename(columns={'最寄駅：距離（分）':'nearest_station','間取り':'floor_type','面積（㎡）':'area','建築年':'builted_year',
            '建ぺい率（％）':'coverrage','容積率（％）':'floor_rate','取引時点':'sold_year','建物の構造':'structure'},inplace = True)
    return data

In [83]:
data = data_processing(df)
data.head()

,nearest_station,floor_type,area,builted_year,structure,coverrage,floor_rate,sold_year,改装,取引価格（総額）_log
0,7,２ＬＤＫ,45,昭和63年,ＲＣ,60.0,150.0,2021年第2四半期,改装済,7.041393
1,4,１Ｋ,15,昭和64年,ＳＲＣ,80.0,400.0,2020年第2四半期,未改装,6.602060
2,18,４ＬＤＫ,80,昭和63年,ＲＣ,60.0,150.0,2020年第4四半期,改装済,7.397940
3,5,２ＬＤＫ,60,平成18年,ＲＣ,80.0,400.0,2020年第1四半期,未改装,7.278754
4,6,３ＤＫ,60,昭和62年,ＳＲＣ,80.0,400.0,2007年第3四半期,未改装,6.929419


In [84]:
data['nearest_station'].unique()

array(['7', '4', '18', '5', '6', '22', nan, '8', '16', '3', '30分?60分',
       '29', '9', '17', '12', '23', '14', '21', '15', '25', '20', '19',
       '10', '13', '11', '1', '24', '1H30?2H', '28', '26', '2', '1H?1H30',
       '27', '0', '2H?'], dtype=object)

In [120]:
def convert_nearest_station(value):
    if value == value:
        if value == '30分?60分':
            return 45
        elif value ==  '1H?1H30':
            return 75
        elif value =='1H30?2H':
            return 105
        elif value =='2H?':
            return 120
        return float(value)

In [121]:
data['nearest_station'] = data['nearest_station'].apply(lambda x: convert_nearest_station(x))

In [122]:
moyori =  data.query('nearest_station=="3"').iloc[3,6]
moyorib =  data.query('nearest_station=="3"').iloc[4,6]
print(moyori == moyori)
print(moyorib == moyorib)

False
True


In [119]:
data.dtypes

nearest_station    float64
floor_type          object
area                object
builted_year        object
structure           object
coverrage          float64
floor_rate         float64
sold_year           object
改装                  object
取引価格（総額）_log       float64
dtype: object